In [1]:
import re
from ast import literal_eval

import pandas as pd
from tqdm.notebook import tqdm
from nltk.corpus import stopwords
from nltk import word_tokenize, ngrams
from IPython.core.display import HTML

In [2]:
tqdm.pandas()

In [3]:
df_train = pd.read_csv("../data/train_v0.7.1.csv",
                       usecols=[
                           'entry_id', 'excerpt', 'age', 'lang',
                           "translation_en", "translation_fr", "translation_es"
                       ])
df_val = pd.read_csv("../data/val_v0.7.1.csv",
                     usecols=[
                         'entry_id', 'excerpt', 'age', 'lang',
                         "translation_en", "translation_fr", "translation_es"
                     ])
df_test = pd.read_csv("../data/test_v0.7.1.csv",
                      usecols=[
                          'entry_id', 'excerpt', 'age', 'lang',
                          "translation_en", "translation_fr", "translation_es"
                      ])

In [4]:
col = "age"
for df in [df_train, df_val, df_test]:
    df[col] = df[col].apply(lambda x: list(sorted(list(set(literal_eval(x))))))

In [5]:
child = 'Children/Youth (5 to 17 years old)'
adult = 'Adult (18 to 59 years old)'
old = 'Older Persons (60+ years old)'
infant = 'Infants/Toddlers (<5 years old)'

In [6]:
df_train.loc[df_train["lang"].eq("en"),
                "translation_en"] = df_train.loc[
                    df_train["lang"].eq("en"), "excerpt"]
# df_train["excerpt"] = df_train["translation_en"]
# df_train = df_train[['entry_id', 'excerpt', 'age', 'lang']]

In [7]:
df_val.loc[df_val["lang"].eq("en"),
                "translation_en"] = df_val.loc[
                    df_val["lang"].eq("en"), "excerpt"]
# df_val["excerpt"] = df_val["translation_en"]
# df_val = df_val[['entry_id', 'excerpt', 'age', 'lang']]

In [8]:
df_test.loc[df_test["lang"].eq("en"),
                "translation_en"] = df_test.loc[
                    df_test["lang"].eq("en"), "excerpt"]
# df_test["excerpt"] = df_test["translation_en"]
# df_test = df_test[['entry_id', 'excerpt', 'age', 'lang']]

In [9]:
def preprocess(doc):
    if doc != doc:
        return ""
    doc = doc.lower()
    # remove preceeding dates
    #doc = re.sub("^\[.+\]", " ", doc).strip()
    #doc = re.sub("^\(.+\)", " ", doc).strip()
    # spaces btw numbers and words
    doc = re.sub('(\d+(\W\d+)?)', r' \1 ', doc).strip()
    doc = re.sub("[‐‑–—―─_]", "-", doc)
    doc = re.sub("(\w)\- (\w)", r"\1\2", doc)
    doc = re.sub(
        "[" + re.escape(
            '_@^~.()[],"“’…<❖‐»—─|•&{≥➢\ue0e4\uf0d8\uf0fc●°#\u200b>`?�€■!‘%;̧\'›«”:≤―\uf0b7$}*´=‑▪\xad❑·–'
        ) + "]", " ", doc)
    #remove some puncs
    doc = re.sub('\s+', " ", doc).strip()
    return doc
##
def preprocess_and_tokenize(doc, n=1):
    doc = preprocess(doc)
    # tokenize
    words = word_tokenize(doc)
    if n == 1:
        return set(words)
    return set(ngrams(words, n))

In [10]:
kw_en_infant = [
#     '(24-59m)',
#     ('0', '-', '59', 'months'),
#     ('0-59', 'months'),
#     ('1', '-year-old'),
#     '1-year-old',
#     ('12-15', 'month'),
#     ('2', '-year-old'),
#     '2-year-old',
#     ('24-59', 'm'),
#     ('3', '-year-old'),
#     '3-year-old',
#     ('4', '-year-old'),
#     '4-year-old',
#     ('5', '-year-old'),
#     '5-year-old',
#     ('6', '-', '23', 'months', 'old'),
#     ('6-', '23', 'months', 'old'),
#     ('6-23', 'months', 'old'),
#     ('age', 'of', '5'),
#     ('age', 'of', 'five'),
#     ('aged', '0', '-', '23', 'months'),
#     ('aged', '0-23', 'months'),
#     ('aged', '0-', '23', 'months'),
#     ('aged', 'zero', '-', 'twenty-three', 'months'),
    'babies',
    'baby',
    #'born',
    #'breastfed',
    ('children', '6-', '59', 'months', 'of', 'age'),
    ('children', '6-59', 'months'),
    ('children', 'under', 'the', 'age', 'of', 'five', 'years'),
    ('children', 'below', 'five', 'years', 'of', 'age'),
    ('children', '6-', '59', 'months'),
    ('children', '0-23', 'months'),
    ('children', 'aged', '0-', '23', 'months'),
    ('children', 'under', '5', 'years', 'of', 'age'),
    ('children', 'of', '6-', '59', 'months'),
    ('children', 'six', '-', 'fifty-nine', 'months'),
    ('children', 'aged', '0-23', 'months'),
    ('children', 'aged', '0', '-', '23', 'months'),
    ('children', 'under', 'five', 'years', 'of', 'age'),
    ('children', 'aged', 'six', 'to', 'fifty-nine', 'months'),
    ('children', 'between', 'the', 'ages', 'of', 'twenty-four', 'and',
     'fifty-nine', 'months'),
    ('children', 'aged', '6-59', 'months'),
    ('children', '6-59', 'months', 'of', 'age'),
    ('children', 'of', '6-59', 'months'),
    ('children', 'under', 'the', 'age', 'of', '5'),
    ('children', 'below', '5', 'years', 'of', 'age'),
    ('children', 'between', 'six', 'to', 'twenty-three', 'months'),
    ('children', 'from', '6', 'to', '59', 'months'),
    ('children', '6', '-', '59', 'months'),
    ('children', 'under', '5', 'years', 'old'),
    ('children', 'under', 'the', 'age', 'of', '5', 'years'),
    ('children', 'aged', 'six', '-', 'fifty-nine', 'months'),
    ('children', 'aged', '6-', '59', 'months'),
    ('children', 'six', '-', 'fifty-nine', 'months', 'of', 'age'),
    ('children', 'aged', '6', '-', '59', 'months'),
    ('children', '0', '-', '23', 'months'),
    ('children', 'from', 'six', 'to', 'fifty-nine', 'months'),
    ('children', 'zero', '-', 'twenty-three', 'months'),
    ('children', 'of', '6', '-', '59', 'months'),
    ('children', 'under', 'five', 'years'),
    ('children', 'of', 'six', '-', 'fifty-nine', 'months'),
    ('children', 'aged', 'zero', '-', 'twenty-three', 'months'),
    ('children', '0-', '23', 'months'),
    ('children', 'between', 'the', 'ages', 'of', '24', 'and', '59', 'months'),
    ('children', 'under', 'the', 'age', 'of', 'five'),
    ('children', '6', '-', '59', 'months', 'of', 'age'),
    ('children', 'under', 'five', 'years', 'old'),
    ('children', 'aged', '6', 'to', '59', 'months'),
    ('children', 'between', '6', 'to', '23', 'months'),
    #'cmam',
    #'congenital',
#     ('infant', 'and', 'young', 'child'),
#     'infant',
#     ('infant', 'and', 'young', 'child', 'feeding'),
#     'infantile',
#     'infants',
#     'iycf',
#     ('less', 'than', 'five', 'years'),
#     ('less', 'than', '5', 'years'),
#     ('live', 'births'),
#     ('live', 'birth'),
#     'measles',
#     'neonatal',
#     'newborn',
#     'newborns',
#     'perinatal',
#     'post-natal',
    ('six', '-', 'twenty-three', 'months', 'old'),
#     'stunted',
#     'stunting',
    ('under', 'the', 'age', 'of', 'five', 'years'),
    ('under', 'the', 'age', 'of', '5'),
    ('under', 'the', 'age', 'of', 'five'),
    ('under', 'the', 'age', 'of', '5', 'years'),
    ("aged", "0-9", "years"),
]
# list(
#     sorted(list(set(kw_en_infant)),
#            key=lambda x: x if isinstance(x, str) else x[0]))
negative_kw_infant = [
    ("women", "with", "babies"),
]
##
r_0_59 = r"\b(5[0-9]|[0-9])\b"  # 0->59
r_0_5 = r"\b([0-5])\b"  # 0->5
##
infant_patterns = [
    #f"less than {r_0_5} years",
    #f"younger than {r_0_5} years",
    #f"under the age of {r_0_5}",
    #f"below the age of {r_0_5}",
    #f"{r_0_59} ?- ?{r_0_59} months old",
    f"aged {r_0_5} ?- ?[0-9] years",
    f"aged {r_0_59} ?- ?{r_0_59} months?",
    #f"{r_0_59} ?- ?{r_0_59} ?m",
    #f"{r_0_59} ?- ?{r_0_59} ?months?",
    #f"{r_0_5}-year-old",
    f"children {r_0_59} ?- ?{r_0_59} months?",
    f"children of {r_0_59} ?- ?{r_0_59} months?",
    f"children under {r_0_5}",
    f"children under the age of {r_0_5}",
    f"children under {r_0_5} years",
    f"children below {r_0_5} years",
    f"children below the age of {r_0_5}",
    f"children below {r_0_5}",
    f"children aged {r_0_59} ?- ?{r_0_59} months?",
    f"children aged {r_0_59} to {r_0_59} months?",
    f"children between {r_0_59} ?- ?{r_0_59} months?",
    f"children between {r_0_59} to {r_0_59} months?",
    f"children between the ages of {r_0_59} and {r_0_59} months?",
    f"children from {r_0_59} ?- ?{r_0_59} months?",
    f"children from {r_0_59} to {r_0_59} months?",
    f"children of {r_0_59} ?- ?{r_0_59} months?",
]


##
def is_infant(row):
    if infant in row["age"]:
        return True
    for kw in negative_kw_infant:
        if isinstance(kw, str) and kw in row['tokenized_excerpt']: return False
        elif len(kw) == 2 and kw in row['bigram_excerpt']: return False
        elif len(kw) == 3 and kw in row['trigram_excerpt']: return False
        elif len(kw) == 4 and kw in row['fourgram_excerpt']: return False
        elif len(kw) == 5 and kw in row['fivegram_excerpt']: return False
        elif len(kw) == 6 and kw in row['sixgram_excerpt']: return False
        elif len(kw) == 7 and kw in row['sevengram_excerpt']: return False
    for p in infant_patterns:
        if re.search(p, row["excerpt_pp"]): return True
    for kw in kw_en_infant:
        if isinstance(kw, str) and kw in row['tokenized_excerpt']: return True
        elif len(kw) == 2 and kw in row['bigram_excerpt']: return True
        elif len(kw) == 3 and kw in row['trigram_excerpt']: return True
        elif len(kw) == 4 and kw in row['fourgram_excerpt']: return True
        elif len(kw) == 5 and kw in row['fivegram_excerpt']: return True
        elif len(kw) == 6 and kw in row['sixgram_excerpt']: return True
        elif len(kw) == 7 and kw in row['sevengram_excerpt']: return True
    return False

In [11]:
kw_en_child = [
    ('14', 'years'), ('17', 'years'), 'adolescent', 'adolescents',
    ('age', 'group', '5', '-', '14', 'years'),
    ('age', 'group', '5-14', 'years'), ('aged', 'ten'), ('aged', '10'),
    ('attending', 'school'), ('below', '18', 'years'), ('below', '18'),
    ('below', 'the', 'age', 'of', '14'), ('below', '14'),
    ('below', '18', 'years', 'of', 'age'), ('below', 'the', 'age', 'of', '17'),
    ('below', 'the', 'age', 'of','18'), ('below', '17'),
    ('below', 'eighteen', 'years'), ('below', 'eighteen'),
    ('below', 'the', 'age', 'of','19'), #'boy', 'boys', 
    'child',
    ('child', 'marriages'), ('child', 'labor'), ('child', 'abuse'),
    ('child', 'marriage'), ('child', 'labour'), ('child', 'friendly'),
    'child-friendly', ('child-friendly', 'spaces'), 'child-headed',
    ('children', 'with', 'disabilities'), 'children',
    ('children', 'under', 'eighteen'),
    ('children', 'below', 'the', 'age', 'of', 'twelve'),
    ('children', 'dropping', 'out', 'of', 'school'),
    ('children', 'separated', 'from', 'their', 'parents'),
    ('children', 'below', 'the', 'age', 'of', '12'),
    ('children', 'not', 'attending', 'school'), ('children', 'school'),
    ('children', 'under', '18'), ('children', 'caregivers'),
    ('early', 'marriage'),
    ('eighteen', 'years'), ('for', 'refugee', 'children'),
    ('for', 'displaced', 'children'), ('fourteen', 'years'), #'girl', 'girls',
    ('girls', 'children'), 'grades', 'in-school', ('labor', 'child'),
    ('minor', 'child'), ('minor', 'children'), 'minors', ('mixed', 'school'),
    ('out', 'of', 'education'), ('out', 'of', 'school'),
    ('out-of-school', 'children'), 'out-of-school', 'pediatric',
    ('primary', 'students'), ('primary', 'student'), ('refugee', 'children'),
    ('rohingya', 'children'), #('school', 'students'),
    ('school', 'aged', 'children'), ('school', 'children'),
    ('school', 'aged'), 'school-age', ('school-age', 'children'),
    ('school-aged', 'children'), 'school-aged', 'school-going', 'schoolboys',
    'schoolchildren', 'schoolgirls', ('separated', 'children'),
    ('seventeen', 'years'), 'teenagers', 'uasc', ('under', '18'),
    ('under', 'the', 'age', 'of','14'), ('under', '18', 'years'), ('under', '14'),
    ('under', 'the', 'age', 'of', 'eighteen'), ('under', 'the', 'age', 'of', '17'),
    ('under', '17'), ('under', 'the', 'age', 'of', '18'),
    ('under', 'the', 'age', 'of', '19'), ('under', 'eighteen'),
    ('under', 'eighteen', 'years'), ('under', '18', 'years', 'of', 'age'),
    ('under', 'the', 'age', 'of', '18'), ('venezuelan', 'minors'),
    ('violence', 'against', 'children'), ('vulnerable', 'children'),
    ('young', 'girls'), ("teenage","girl"), ("teenage","girls"),
    ("teenage","boy"), ("teenage","boys"), ("older", "than", "5", "years")
]
# list(
#     sorted(list(set(kw_en_child)),
#            key=lambda x: x if isinstance(x, str) else x[0]))
negative_kw_child = [
    "gam",
    "sam",
    "muac",
    "24-59m",
    ('above', 'the', 'age', '18', 'years'),
    ('above', '18', 'years', 'of', 'age'),
    ('moderate', 'acute', 'malnutrition', 'mam'),
    ('moderate', 'acute', 'malnutrition'),
    'mam',
    ('aged', '6-59', 'months'),
    ('aged', '6-59'),
]
##
##
r_6_18 = r"\b(1[0-8]|[6-9])\b"  # 6->18
r_5_15 = r"\b(1[0-5]|[5-9])\b"  # 5->15
r_5_18 = r"\b(1[0-8]|[5-9])\b"  # 5->18
r_14_18 = r"\b(1[4-8])\b"  # 4->18
##
child_patterns = [
    f"between {r_6_18} ?- ?{r_6_18}",
    f"between {r_6_18} to {r_6_18}",
    f"between the age of {r_6_18} and {r_6_18}",
    f"the {r_6_18} ?- ?{r_6_18} age",
    f"{r_6_18} ?- ?{r_6_18} years",
    f"{r_6_18} to {r_6_18} years",
    f"{r_6_18} ?-year-old",
    f"{r_6_18} ?-year-olds",
    f"{r_6_18} ?- ?{r_6_18} years old",
    f"{r_6_18} ?- ?{r_6_18} age",
    f"ages of {r_6_18} ?- ?{r_6_18}",
    f"ages of {r_6_18} and {r_6_18}",
    f"age of {r_6_18} ?- ?{r_6_18}",
    f"age of {r_6_18} years",
    f"age {r_6_18}",
    f"aged {r_6_18}",
    f"age {r_6_18} ?- ?{r_6_18}",
    f"ages {r_6_18} ?- ?{r_6_18}",
    f"aged {r_6_18} ?- ?{r_6_18}",
    f"age {r_6_18} ?- ?{r_6_18}",
    f"{r_6_18} years",
    f"age group {r_5_18} ?- ?{r_6_18} years",
    f"below {r_14_18} years",
    f"below the age of {r_14_18}",
    f"children under {r_14_18}",
    f"under the age of {r_14_18}",
    f"under {r_14_18} years",
    f"below {r_14_18} years",
    f"below the age of {r_14_18}",
    f"younger than {r_14_18}",
    f"less than {r_14_18}",
    f"older than {r_5_15} years",
]
##
def is_child(row):
    for kw in negative_kw_child:
        if isinstance(kw, str) and kw in row['tokenized_excerpt']: return False
        elif len(kw) == 2 and kw in row['bigram_excerpt']: return False
        elif len(kw) == 3 and kw in row['trigram_excerpt']: return False
        elif len(kw) == 4 and kw in row['fourgram_excerpt']: return False
        elif len(kw) == 5 and kw in row['fivegram_excerpt']: return False
        elif len(kw) == 6 and kw in row['sixgram_excerpt']: return False
        elif len(kw) == 7 and kw in row['sevengram_excerpt']: return False
    for p in child_patterns:
        if re.search(p, row["excerpt_pp"]): return True
    for kw in kw_en_child:
        if isinstance(kw, str) and kw in row['tokenized_excerpt']: return True
        elif len(kw) == 2 and kw in row['bigram_excerpt']: return True
        elif len(kw) == 3 and kw in row['trigram_excerpt']: return True
        elif len(kw) == 4 and kw in row['fourgram_excerpt']: return True
        elif len(kw) == 5 and kw in row['fivegram_excerpt']: return True
        elif len(kw) == 6 and kw in row['sixgram_excerpt']: return True
        elif len(kw) == 7 and kw in row['sevengram_excerpt']: return True
    return False

In [12]:
kw_en_adult = [
    #'adult', 'adults', 
    #('asylum', 'seekers'), 'breadwinner',
    #('care', 'givers'), 'care-giving', 'caregivers', 'child-bearing',
    #'childbearing', 'employability', 'employed', 'employees', 'employers',
    #'employment', 'entrepreneurs', ('female', 'detainees'), 'female-headed',
    #('foreign', 'health', 'professionals'), 'gbv', 'graduates',
    #('had', 'given', 'a', 'life', 'birth'), 'husbands', 'jobs',
    #('killing', 'of', 'teachers'), ('labor', 'market'), ('life', 'birth'),
    #('male', 'headed'), 'male-headed', 'maternal', 'men', 'mothers', 'parents',
    #('reproductive', 'age'), 
    #'scholarships', ('stateless', 'persons'),
    #('to', 'work'), 'underemployed', 'underemployment', 'unemployed', 'unemployment',
    #('unintended', 'pregnancies'), 'university', 'widow',
    #'widows', 'wives', 
    #'woman', 'women', 'women-headed', 'workers',
    #('working', 'in'), 
    'working-age', 
    'youth', 'youths'
]
# list(
#     sorted(list(set(kw_en_adult)),
#            key=lambda x: x if isinstance(x, str) else x[0]))
negative_kw_adult = []
##
r_18_59 = r"\b([2-5][0-9]|1[8-9])\b"  # 18->59
r_20_59 = r"\b([2-5][0-9])\b"  # 20->59
r_15_19 = r"\b(1[5-9])\b"  # 15->19
r_11_19 = r"\b(1[1-9])\b"  # 11->19
r_18_49 = r"\b([2-4][0-9]|1[8-9])\b"  # 18->49
##
adult_patterns = [
    f"between {r_11_19} ?- ?{r_20_59}",
    f"between {r_18_59} ?- ?{r_18_59}",
    f"between {r_18_59} to {r_18_59}",
    f"between {r_11_19} to {r_20_59}",
    f"between the age of {r_18_59} and {r_18_59}",
    f"between the age of {r_11_19} and {r_20_59}",
    f"the {r_18_59} ?- ?{r_18_59} age",
    f"{r_18_59} ?- ?{r_18_59} years",
    f"{r_18_59} to {r_18_59} years",
    f"{r_18_59}\+ years",
    f"{r_18_59} ?-year-old",
    f"{r_18_59} ?-year-olds",
    f"{r_18_59} ?- ?{r_18_59} years old",
    f"{r_18_59} ?- ?{r_18_59} age",
    f"ages of {r_18_59} ?- ?{r_18_59}",
    f"ages of {r_11_19} ?- ?{r_20_59}",
    f"ages of {r_11_19} and {r_20_59}",
    f"age of {r_18_59} ?- ?{r_18_59}",
    f"age of {r_18_59} ?- ?{r_18_59}",
    f"age of {r_18_59} years",
    f"age {r_18_59}",
    f"age {r_18_59} ?- ?{r_18_59}",
    f"ages {r_11_19} ?- ?{r_20_59}",
    f"aged {r_18_59} ?- ?{r_18_59}",
    f"age {r_11_19} ?- ?{r_20_59}",
    f"\d\d+,?\d* female",
    f"\d\d+,?\d* females",
    f"\d\d+,?\d* male",
    f"\d\d+,?\d* males",
    #f"\d\d+,?\d* people",
    #f"\d\d+,?\d* persons",
    f"older than {r_18_49} years",
    f"{r_18_49} years and above",
    f"{r_18_49} years old and above",
    f"above the age of {r_18_49}",
    f"over the age of {r_18_49}",
    f"over >{r_18_59} ?years",
    f"above >{r_18_59} ?years",
]
##
def is_adult(row):
    for kw in negative_kw_adult:
        if isinstance(kw, str) and kw in row['tokenized_excerpt']: return False
        elif len(kw) == 2 and kw in row['bigram_excerpt']: return False
        elif len(kw) == 3 and kw in row['trigram_excerpt']: return False
        elif len(kw) == 4 and kw in row['fourgram_excerpt']: return False
        elif len(kw) == 5 and kw in row['fivegram_excerpt']: return False
        elif len(kw) == 6 and kw in row['sixgram_excerpt']: return False
        elif len(kw) == 7 and kw in row['sevengram_excerpt']: return False
    for p in adult_patterns:
        if re.search(p, row["excerpt_pp"]): return True
    for kw in kw_en_adult:
        if isinstance(kw, str) and kw in row['tokenized_excerpt']: return True
        elif len(kw) == 2 and kw in row['bigram_excerpt']: return True
        elif len(kw) == 3 and kw in row['trigram_excerpt']: return True
        elif len(kw) == 4 and kw in row['fourgram_excerpt']: return True
        elif len(kw) == 5 and kw in row['fivegram_excerpt']: return True
        elif len(kw) == 6 and kw in row['sixgram_excerpt']: return True
        elif len(kw) == 7 and kw in row['sevengram_excerpt']: return True
    return False

In [13]:
kw_en_old = [
    ('55', 'years', 'and', 'above'),
    ('59', 'years', 'and', 'above'),
    ('60', 's'),
    ('60', 'years'),
    ('60', 'years', 'and', 'above'),
    '60-year-old',
    ('62', 'older'),
    ('65', 'years'),
    ('65', 'years', 'and', 'above'),
    ('70', 'years'),
    ('70', 'years', 'and', 'above'),
    ('70', '-year-old'),
    '70-year-old',
    ('75', 'years', 'and', 'above'),
    ('80', 'years'),
    '80-year-old',
    ('above', '65', 'years'),
    ('above', '65', 'years', 'of', 'age'),
    ('above', '60', 'years'),
    ('above', '55', 'years'),
    ('above', '70'),
    ('above', '80'),
    ('above', '59'),
    ('above', '70', 'years'),
    ('above', '59', 'years'),
    ('above', '75', 'years'),
    ('above', '60'),
    ('above', '65'),
    ('adults', 'aged', '65'),
    ('adults', 'aged', '70'),
    ('aged', 'above', '59'),
    ('aged', '80', 'and', 'above'),
    ('aged', 'above', '60'),
    ('aged', '65'),
    ('aged', 'above', '85'),
    ('aged', 'above', '55'),
    ('aged', '70', 'and', 'above'),
    ('aged', '75', 'and', 'above'),
    ('aged', 'above', '65'),
    ('aged', '80'),
    ('aged', 'above', '75'),
    ('aged', '70'),
    ('aged', 'above', '50'),
    ('aged', 'above', '70'),
    ('aged', '60', 'and', 'above'),
    ('aged', '65', 'and', 'above'),
    ('aged', '60'),
    ('aged', 'above', '80'),
    #'ageing',
    ('elderly', 'community'),
    ('elderly', 'population'),
    ('elderly', 'men'),
    ('elderly', 'women'),
    ('elderly', 'persons'),
    'elderly',
    ('elderly', 'people'),
    ('elders', 'people'),
    ('older', 'persons'),
    ('older', 'people'),
    ('older', 'population'),
    ('older', 'person'),
    ('older', 'men'),
    ('older', 'women'),
    ('over', '59'),
    ('over', '70', 'years'),
    ('over', '59', 'years'),
    ('over', '55'),
    ('over', '60', 'years', 'old'),
    ('over', '80', 'years', 'old'),
    ('over', '60'),
    ('over', '65', 'years'),
    ('over', '60', 'years'),
    ('over', '55', 'years'),
    ('over', '80', 'years'),
    ('people', 'aged', '59'),
    ('people', 'over', '65'),
    ('people', 'aged', '55'),
    ('people', 'aged', '60'),
    ('people', 'over', '80'),
    ('people', 'over', '70'),
    ('the', '59', '+', 'population'),
    "elders",
    ('50', '+', 'years'),
    ("geriatric", "population"),
]
# list(
#     sorted(list(set(kw_en_old)),
#            key=lambda x: x if isinstance(x, str) else x[0]))
negative_kw_old = []
r_59_100 = r"\b(100|[6-9][0-9]|59)\b"  # 59-100
r_50_100 = r"\b(100|[6-9][0-9]|5[0-9])\b"  # 50-100
r_61_100 = r"\b(100|[6-9][1-9])\b"  # 61-100
r_60_100 = r"\b(100|[6-9][0-9])\b"  # 60-100
r_0_59 = r"\b(5[0-9]|[0-9][0-9])\b"  # 0-59
patterns_old = [
    f"between {r_59_100} ?- ?{r_59_100}",
    f"between {r_50_100} ?- ?{r_61_100}",
    f"{r_59_100} years and older",
    f"{r_59_100} years and above",
    f"{r_59_100} years old and above",
    f"above {r_59_100} years",
    f"over {r_59_100} years",
    f"above the age of {r_50_100}",
    f"over the age of {r_50_100}",
    f"between {r_50_100} ?- ?{r_59_100}",
    f"between {r_50_100} to {r_59_100}",
    f"between the age of {r_50_100} and {r_59_100}",
    f"between the ages of {r_50_100} and {r_59_100}",
    f"the {r_59_100} ?- ?{r_59_100} age",
    f"{r_50_100} ?- ?{r_61_100} years",
    f"{r_50_100} to {r_61_100} years",
    f"{r_50_100}\+ years",
    f"{r_60_100} ?-year-old",
    f"{r_60_100} ?-year-olds",
    f"{r_50_100} ?- ?{r_61_100} years old",
    f"{r_50_100} ?- ?{r_61_100} age",
    f"ages of {r_50_100} ?- ?{r_61_100}",
    f"ages of {r_50_100} ?- ?{r_61_100}",
    f"ages of {r_50_100} and {r_61_100}",
    f"age of {r_50_100} ?- ?{r_61_100}",
    f"age of {r_50_100} ?- ?{r_61_100}",
    f"age of {r_60_100} years",
    f"age {r_60_100}",
    f"age {r_50_100} ?- ?{r_61_100}",
    f"ages {r_50_100} ?- ?{r_61_100}",
    f"aged {r_50_100} ?- ?{r_61_100}",
    f"age {r_50_100} ?- ?{r_61_100}",
    f"aged {r_50_100} years and older",
    f"older than {r_50_100}",
]


##
def is_old(row):
    for kw in negative_kw_old:
        if isinstance(kw, str) and kw in row['tokenized_excerpt']: return False
        elif len(kw) == 2 and kw in row['bigram_excerpt']: return False
        elif len(kw) == 3 and kw in row['trigram_excerpt']: return False
        elif len(kw) == 4 and kw in row['fourgram_excerpt']: return False
        elif len(kw) == 5 and kw in row['fivegram_excerpt']: return False
        elif len(kw) == 6 and kw in row['sixgram_excerpt']: return False
        elif len(kw) == 7 and kw in row['sevengram_excerpt']: return False
    for p in patterns_old:
        if re.search(p, row["excerpt_pp"]): return True
    for kw in kw_en_old:
        if isinstance(kw, str) and kw in row['tokenized_excerpt']: return True
        elif len(kw) == 2 and kw in row['bigram_excerpt']: return True
        elif len(kw) == 3 and kw in row['trigram_excerpt']: return True
        elif len(kw) == 4 and kw in row['fourgram_excerpt']: return True
        elif len(kw) == 5 and kw in row['fivegram_excerpt']: return True
        elif len(kw) == 6 and kw in row['sixgram_excerpt']: return True
        elif len(kw) == 7 and kw in row['sevengram_excerpt']: return True
    return False

In [14]:
# preprocess
df_train["excerpt_pp"] = df_train["translation_en"].progress_apply(preprocess)
# tokenize and cacl ngrams
df_train["tokenized_excerpt"] = df_train["translation_en"].progress_apply(preprocess_and_tokenize)
df_train["bigram_excerpt"] = df_train["translation_en"].progress_apply(lambda x: preprocess_and_tokenize(x, 2))
df_train["trigram_excerpt"] = df_train["translation_en"].progress_apply(lambda x: preprocess_and_tokenize(x, 3))
df_train["fourgram_excerpt"] = df_train["translation_en"].progress_apply(lambda x: preprocess_and_tokenize(x, 4))
df_train["fivegram_excerpt"] = df_train["translation_en"].progress_apply(lambda x: preprocess_and_tokenize(x, 5))
df_train["sixgram_excerpt"] = df_train["translation_en"].progress_apply(lambda x: preprocess_and_tokenize(x, 6))
df_train["sevengram_excerpt"] = df_train["translation_en"].progress_apply(lambda x: preprocess_and_tokenize(x, 7))

  0%|          | 0/126323 [00:00<?, ?it/s]

  0%|          | 0/126323 [00:00<?, ?it/s]

  0%|          | 0/126323 [00:00<?, ?it/s]

  0%|          | 0/126323 [00:00<?, ?it/s]

  0%|          | 0/126323 [00:00<?, ?it/s]

  0%|          | 0/126323 [00:00<?, ?it/s]

  0%|          | 0/126323 [00:00<?, ?it/s]

  0%|          | 0/126323 [00:00<?, ?it/s]

In [15]:
# preprocess
df_val["excerpt_pp"] = df_val["translation_en"].progress_apply(preprocess)
# tokenize and cacl ngrams
df_val["tokenized_excerpt"] = df_val["translation_en"].progress_apply(preprocess_and_tokenize)
df_val["bigram_excerpt"] = df_val["translation_en"].progress_apply(lambda x: preprocess_and_tokenize(x, 2))
df_val["trigram_excerpt"] = df_val["translation_en"].progress_apply(lambda x: preprocess_and_tokenize(x, 3))
df_val["fourgram_excerpt"] = df_val["translation_en"].progress_apply(lambda x: preprocess_and_tokenize(x, 4))
df_val["fivegram_excerpt"] = df_val["translation_en"].progress_apply(lambda x: preprocess_and_tokenize(x, 5))
df_val["sixgram_excerpt"] = df_val["translation_en"].progress_apply(lambda x: preprocess_and_tokenize(x, 6))
df_val["sevengram_excerpt"] = df_val["translation_en"].progress_apply(lambda x: preprocess_and_tokenize(x, 7))

  0%|          | 0/14425 [00:00<?, ?it/s]

  0%|          | 0/14425 [00:00<?, ?it/s]

  0%|          | 0/14425 [00:00<?, ?it/s]

  0%|          | 0/14425 [00:00<?, ?it/s]

  0%|          | 0/14425 [00:00<?, ?it/s]

  0%|          | 0/14425 [00:00<?, ?it/s]

  0%|          | 0/14425 [00:00<?, ?it/s]

  0%|          | 0/14425 [00:00<?, ?it/s]

In [16]:
# preprocess
df_test["excerpt_pp"] = df_test["translation_en"].progress_apply(preprocess)
# tokenize and cacl ngrams
df_test["tokenized_excerpt"] = df_test["translation_en"].progress_apply(preprocess_and_tokenize)
df_test["bigram_excerpt"] = df_test["translation_en"].progress_apply(lambda x: preprocess_and_tokenize(x, 2))
df_test["trigram_excerpt"] = df_test["translation_en"].progress_apply(lambda x: preprocess_and_tokenize(x, 3))
df_test["fourgram_excerpt"] = df_test["translation_en"].progress_apply(lambda x: preprocess_and_tokenize(x, 4))
df_test["fivegram_excerpt"] = df_test["translation_en"].progress_apply(lambda x: preprocess_and_tokenize(x, 5))
df_test["sixgram_excerpt"] = df_test["translation_en"].progress_apply(lambda x: preprocess_and_tokenize(x, 6))
df_test["sevengram_excerpt"] = df_test["translation_en"].progress_apply(lambda x: preprocess_and_tokenize(x, 7))

  0%|          | 0/17200 [00:00<?, ?it/s]

  0%|          | 0/17200 [00:00<?, ?it/s]

  0%|          | 0/17200 [00:00<?, ?it/s]

  0%|          | 0/17200 [00:00<?, ?it/s]

  0%|          | 0/17200 [00:00<?, ?it/s]

  0%|          | 0/17200 [00:00<?, ?it/s]

  0%|          | 0/17200 [00:00<?, ?it/s]

  0%|          | 0/17200 [00:00<?, ?it/s]

In [17]:
INFANT = 'Infants/Toddlers (<5 years old)'
CHILD = 'Children/Youth (5 to 17 years old)'
ADULT = 'Adult (18 to 59 years old)'
OLD = 'Older Persons (60+ years old)'

In [18]:
def pred_age(row):
    lbls = []
    if is_infant(row):
        lbls.append(INFANT)
    if is_child(row):
        lbls.append(CHILD)
    if is_adult(row):
        lbls.append(ADULT)
    if is_old(row):
        lbls.append(OLD)
    return lbls

In [19]:
# i = 11255
# row = df_test[df_test['entry_id'].eq(i)].iloc[0]
# row["excerpt"]

In [20]:
# is_child(row)

In [21]:
df_train["age_kw_pred"] = df_train.progress_apply(pred_age, axis=1)

  0%|          | 0/126323 [00:00<?, ?it/s]

In [22]:
df_val["age_kw_pred"] = df_val.progress_apply(pred_age, axis=1)

  0%|          | 0/14425 [00:00<?, ?it/s]

In [23]:
df_test["age_kw_pred"] = df_test.progress_apply(pred_age, axis=1)

  0%|          | 0/17200 [00:00<?, ?it/s]

In [24]:
df_train.to_csv("train_0.7.1_keyword_age.csv",
               index=False,
               columns=['entry_id', 'age_kw_pred'])
##
df_val.to_csv("val_0.7.1_keyword_age.csv",
               index=False,
               columns=['entry_id', 'age_kw_pred'])
##
df_test.to_csv("test_0.7.1_keyword_age.csv",
               index=False,
               columns=['entry_id', 'age_kw_pred'])

In [25]:
(df_train["age_kw_pred"] == df_train["age"]).sum(), df_train.shape

(108248, (126323, 16))

In [26]:
(df_val["age_kw_pred"] == df_val["age"]).sum(), df_val.shape

(12338, (14425, 16))

In [28]:
(df_test["age_kw_pred"] == df_test["age"]).sum(), df_test.shape

(14610, (17200, 16))